In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [23]:
import os
path = '/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

calculate.py		 Registration_test_jurai_ptlk.py  TNUTEJN016.pcd
checkpoint		 Registration_test_jurai.py	  Tokuichi_divide.py
data_list_key_0.csv	 Registration_test.py		  T-pipe_matching.py
data_list_key_90.csv	 S3DIS.py			  T-pipe_test_jurai_ptlk.py
data_list_key_L_180.csv  sensor_cheese_noise.pcd	  Tpipe_test_jurai_ptlk_time.ipynb
data_list_key_L_90.csv	 sensor_Ljoint_180_2.pcd	  T-pipe_test_jurai.py
dataset.py		 sensor_Ljoint_90_4.pcd		  Tpipe_test_jurai_time.ipynb
Global_optimizer.py	 sensor_tpip_135_2.pcd		  T-pipe_test.py
learning3d		 sensor_tpip_45_3.pcd		  Tpipe_test_time.ipynb
network.py		 sensor_tpip_90_1.pcd		  WMU2LR2020_half2.pcd
pretrained		 TNUTEJN016_half2.pcd		  WMU2LR2020_half_unnoise.pcd
__pycache__		 TNUTEJN016_half_unnoise.pcd	  WMU2LR2020.pcd


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import h5py
import subprocess
import shlex
import json
import glob
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import minkowski
from scipy.spatial import cKDTree
from scipy.spatial.transform import Rotation
from torch import sin, cos
import open3d as o3d
from tqdm import tqdm
import torchvision
import logging
import random
import os
import numpy.linalg as LA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import copy
import time
import gc

In [5]:
############################################
# 位置合わせ描画
############################################
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def prepare_dataset(voxel_size, target, source):

    #draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh


In [6]:

def rotation_matrix_to_euler_angles(R):
    # Extract angles using trigonometric relations
    roll = np.arctan2(-R[1, 2], R[2, 2])
    pitch = np.arctan2(-(R[0, 2]*np.cos(roll)), R[2, 2])
    yaw = np.arctan2(-R[0, 1], R[0, 0])
    #yaw = np.arctan2(-1, -1)

    return np.array([roll, pitch, yaw])

def coordinate_3d(axes, range_x, range_y, range_z, grid = True):
    axes.set_xlabel("x", fontsize = 14)
    axes.set_ylabel("y", fontsize = 14)
    axes.set_zlabel("z", fontsize = 14)
    axes.set_xlim(range_x[0], range_x[1])
    axes.set_ylim(range_y[0], range_y[1])
    axes.set_zlim(range_z[0], range_z[1])
    if grid == True:
        axes.grid()

def visual_vector_3d(axes, loc, vector, color = "red"):
    axes.quiver(loc[0], loc[1], loc[2],
              vector[0], vector[1], vector[2],
              color = color, lw=3)

###################
#重心計算
###################
def calculate_centroid(pointcloud):
    # Calculate the centroid (mean) of the point cloud along each axis
    centroid = np.mean(pointcloud, axis=0)
    return centroid

def translate_to_origin(pointcloud):
    # Calculate the centroid of the point cloud
    centroid = calculate_centroid(pointcloud)

    # Translate the point cloud to move the centroid to the origin
    translated_pointcloud = pointcloud - centroid

    return translated_pointcloud

###################
#Registration
###################
# ICP registration module.
class ICP:
  def __init__(self, threshold=0.1, max_iteration=100):
    self.threshold = threshold
    self.criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iteration)

  def preprocess(self, template, source):
    if self.is_tensor: template, source = template.detach().cpu().numpy(), source.detach().cpu().numpy()	# Convert to ndarray if tensors.

    if len(template.shape) > 2:
      template, source = template[0], source[0]

    # Find mean of template & source.
    self.template_mean = np.mean(template, axis=0, keepdims=True)
    self.source_mean = np.mean(source, axis=0, keepdims=True)

		# Convert to open3d point clouds.
    template_ = o3d.geometry.PointCloud()
    source_ = o3d.geometry.PointCloud()

		# Subtract respective mean from each point cloud.
    template_.points = o3d.utility.Vector3dVector(template - self.template_mean)
    source_.points = o3d.utility.Vector3dVector(source - self.source_mean)
    return template_, source_

  # Postprocess on transformation matrix.
  def postprocess(self, res):
    est_R = res.transformation[0:3, 0:3]		# ICP's rotation matrix (source -> template)
    t_ = np.array(res.transformation[0:3, 3]).reshape(1, -1)			# ICP's translation vector (source -> template)
    est_T = np.array(res.transformation)								# ICP's transformation matrix (source -> template)
    est_t = np.matmul(est_R, -self.source_mean.T).T + t_ + self.template_mean[0] 	# update predicted translation according to eq. 3
    est_T[0:3, 3] = est_t
    return est_R, est_t, est_T

  # Convert result to pytorch tensors.
  @staticmethod
  def convert2tensor(result):
    if torch.cuda.is_available(): device = 'cuda'
    else: device = 'cpu'
    result['est_R']=torch.tensor(result['est_R']).to(device).float().view(-1, 3, 3) 		# Rotation matrix [B, 3, 3] (source -> template)
    result['est_t']=torch.tensor(result['est_t']).to(device).float().view(-1, 1, 3)			# Translation vector [B, 1, 3] (source -> template)
    result['est_T']=torch.tensor(result['est_T']).to(device).float().view(-1, 4, 4)			# Transformation matrix [B, 4, 4] (source -> template)
    return result

  # icp registration.
  def __call__(self, template, source, start):
    self.is_tensor = torch.is_tensor(template)
    ### 重心移動の計算 ###
    source_a = source.detach().cpu().numpy()
    source_a = source_a[0]
    jusin_mae = np.mean(source_a, axis=0)

    # tensorからo3d
    template, source = self.preprocess(template, source)

    source.paint_uniform_color([0, 1, 0])
    template.paint_uniform_color([0, 0, 1])
    template_vis = template
    source_vis = source
    template_copy = copy.deepcopy(template)
    source_copy = copy.deepcopy(source)

    ### ダウンサンプリング ###
    if theta == 0:
      voxel_size = 0.008     # 0度
    elif theta == 45:
      voxel_size = 0.011     # 45度
    elif theta == 90:
      voxel_size = 0.015     # 90度
    elif theta == 135:
      voxel_size = 0.015     # 180度
    elif theta == "L_90":
      voxel_size = 0.011    # L90度
    elif theta == "L_180":
      voxel_size = 0.011    # L180度

    template = template.voxel_down_sample(voxel_size)
    source = source.voxel_down_sample(voxel_size)

    ### マスクテンプレの主成分分析 ###
    T_trans = np.array(template.points)
    T_cov = T_trans.T @ T_trans
    T_W, T_V_pca = np.linalg.eig(T_cov)
    T_index = T_W.argsort()[::-1]
    T_W = T_W[T_index]
    T_V_pca = T_V_pca[:, T_index]

		### ソースの主成分分析 ###
    S_trans = np.array(source.points)
    S_cov = S_trans.T @ S_trans
    S_W, S_V_pca = np.linalg.eig(S_cov)
    S_index = S_W.argsort()[::-1]
    S_W = S_W[S_index]
    S_V_pca = S_V_pca[:, S_index]

    if np.linalg.det(S_V_pca @ T_V_pca.T) < 0:
      K = np.array([[-1, 0, 0],[0, 1, 0],[0, 0, 1]])     # 0度、45度、90度
      R = S_V_pca @ K @ T_V_pca.T
    else:
      R = S_V_pca @ T_V_pca.T
    ###print(np.linalg.det(R))
		###print(R)
		###転置処理###
    R = R.T

    ### SVDアルゴリズム（概略マッチング） ###
    transformation = np.eye(4)
    for i in range(3):
      for j in range(3):
        transformation[i][j] = R[i][j]
        elapsed_time_1 = time.time() - start

    ### ICPアルゴリズム（精密マッチング） ###
    res = o3d.pipelines.registration.registration_icp(source, template, self.threshold, transformation, criteria=self.criteria)	# icp registration in open3d.

    elapsed_time_2 = time.time() - start

    est_R, est_t, est_T = self.postprocess(res)
    result = {'est_R': est_R,
		          'est_t': est_t,
		          'est_T': est_T}

    if self.is_tensor: result = self.convert2tensor(result)
    return result, elapsed_time_1, elapsed_time_2


# Define Registration Algorithm
def registration_algorithm():
  reg_algorithm = ICP()
  return reg_algorithm


# Register template and source pairs.
class Registration:
	def __init__(self):
		#self.reg_algorithm = reg_algorithm
		#self.is_rpmnet = True if self.reg_algorithm == 'rpmnet' else False
		#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
		self.reg_algorithm = registration_algorithm()

	@staticmethod
	def pc2points(data):
		if len(data.shape) == 3:
			return data[:, :, :3]
		elif len(data.shape) == 2:
			return data[:, :3]

	def register(self, template, source, start):
		# template, source: 		Point Cloud [B, N, 3] (torch tensor)

		# No need to use normals. Only use normals for RPM-Net.
		#if not self.is_rpmnet == 'rpmnet':
		#	template, source = self.pc2points(template), self.pc2points(source)

		result = self.reg_algorithm(template, source, start)
		return result

def pc2open3d(data):
	if torch.is_tensor(data): data = data.detach().cpu().numpy()
	if len(data.shape) == 2:
		pc = o3d.geometry.PointCloud()
		pc.points = o3d.utility.Vector3dVector(data)
		return pc
	else:
		print("Error in the shape of data given to Open3D!, Shape is ", data.shape)

In [68]:
theta = "L_90"

# モデルのロード
save_path = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/checkpoint/model_weight_epoch300_batchsize32_plane.pth"
model_load = torch.load(save_path)

# テンプレートデータのロード
pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/TNUTEJN016.pcd"
# ソースデータのロード
if theta == 0:   ## 0度 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_cheese_noise.pcd"
elif theta == 45:   ## 45度 ##
	pass
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_45_3.pcd"
elif theta == 90:   ## 90度 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_90_1.pcd"
elif theta == 135:   ## 135度 ##
	pass
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_tpip_135_2.pcd"
elif theta == "L_90":   ## L90 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_Ljoint_90_4.pcd"
	pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/WMU2LR2020.pcd"
elif theta == "L_180":   ## L180 ##
	pcd_rot_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/sensor_Ljoint_180_2.pcd"
	pcd_file = "/content/drive/MyDrive/ColabNotebooks/research/M1_object_detection/論文資料/program/WMU2LR2020.pcd"

# テンプレとソースを点群データに変換
pcd_cheese = o3d.io.read_point_cloud(pcd_file)
pcd_cheese_rot = o3d.io.read_point_cloud(pcd_rot_file)

numpy_cheese_points = np.array(pcd_cheese.points)
numpy_cheese_rot_points = np.array(pcd_cheese_rot.points)

list_time1 = []
list_time2 = []
list_time3 = []
list_timeF = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_load.to(device)

points = np.asarray(pcd_cheese_rot.points)
num_points = len(points)
min_bound = np.min(numpy_cheese_rot_points, axis=0)
max_bound = np.max(numpy_cheese_rot_points, axis=0)
box_width = max_bound[0] - min_bound[0]
box_height = max_bound[1] - min_bound[1]
box_depth = max_bound[2] - min_bound[2]
#points_inside = points[(points >= min_bound) & (points <= max_bound)].reshape(-1, 3)
if theta == 0:   ## 0度 ##
	weight = 0.8   # 0度
elif theta == 45:   ## 90度 ##
	weight = 0.8   # 90度
elif theta == 90:   ## 90度 ##
	weight = 0.8   # 90度
elif theta == 135:   ## 90度 ##
	weight = 0.8   # 90度
elif theta == "L_90":   ## 90度 ##
	weight = 0.8   # 90度
elif theta == "L_180":   ## 90度 ##
	weight = 0.8   # 90度

n = 1000
for i in range(n):
	gc.collect()
	torch.cuda.synchronize()
	start = time.time()
	density = num_points / ( box_width * box_height * box_depth )
	voxel_size = weight * (1.0 / density) ** (1/3)
	pcd_cheese = pcd_cheese.voxel_down_sample(voxel_size) #初期値0.006
	numpy_cheese_points = np.array(pcd_cheese.points)
	numpy_cheese_rot_points = np.array(pcd_cheese_rot.points)
	pcd_cheese_points = o3d.geometry.PointCloud()
	pcd_cheese_points.points = o3d.utility.Vector3dVector(numpy_cheese_points)
	pcd_cheese_rot_points = o3d.geometry.PointCloud()
	pcd_cheese_rot_points.points = o3d.utility.Vector3dVector(numpy_cheese_rot_points)

	template_cheese = torch.tensor(numpy_cheese_points, dtype=torch.float32).unsqueeze(0)
	source_cheese = torch.tensor(numpy_cheese_rot_points, dtype=torch.float32).unsqueeze(0)
	template_cheese = template_cheese.to(device)
	source_cheese = source_cheese.to(device)

	# 位置合わせ準備を行う（関数を呼び出す）
	registration_model = Registration()

	with torch.no_grad():
		masked_template_cheese, predicted_mask_cheese = model_load(template_cheese, source_cheese)
	elapsed_time_1 = time.time() - start

	# 提案手法（MaskNet、SVD、ICP）の実行（実際のデータを代入）
	result_cheese = registration_model.register(masked_template_cheese, source_cheese, start)
	est_T_cheese = result_cheese[0]['est_T']     # est_T：ICPの変換行列

	#####計測終了#####
	torch.cuda.synchronize()
	elapsed_time_F = time.time() - start
	elapsed_time_2 = result_cheese[1]
	elapsed_time_3 = result_cheese[2]
	list_time1.append(elapsed_time_1)
	list_time2.append(elapsed_time_2)
	list_time3.append(elapsed_time_3)
	list_timeF.append(elapsed_time_F)


In [73]:
elapsed_time_1

0.0063800811767578125

In [74]:
elapsed_time_2

0.00875234603881836

In [75]:
elapsed_time_3

0.009119749069213867

In [76]:
elapsed_time_F

0.009614229202270508

In [77]:
import csv
def save_list_to_csv(filename, data):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Data'])
        for item in data:
            writer.writerow([item])
# リストの中身をCSVファイルに保存
save_list_to_csv(f'data_list_proposed_{theta}.csv', list_timeF)